In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
!pip install keras

In [10]:
import keras
import keras.layers as L

In [11]:
points_data = pd.read_csv("00_gt/gt.csv")
points_data.head()

,filename,x1,y1,x2,y2,x3,y3,x4,y4,x5,...,x10,y10,x11,y11,x12,y12,x13,y13,x14,y14
0,00000.jpg,51,51,127,60,169,65,203,52,68,...,193,84,156,144,92,174,140,180,168,179
1,00001.jpg,21,20,36,18,57,19,80,25,23,...,78,33,43,46,31,61,45,66,60,65
2,00002.jpg,23,116,39,83,47,57,56,19,36,...,62,31,91,86,106,122,115,100,121,81
3,00003.jpg,29,18,63,25,84,32,105,34,38,...,95,45,72,71,46,83,63,87,75,89
4,00004.jpg,31,53,76,44,107,38,136,33,40,...,129,49,111,105,72,137,106,132,126,121


In [12]:
size_data = pd.read_csv("00_gt/img_shapes.csv")
size_data.head()

,img_filename,n_rows,n_cols
0,00000.jpg,262,262
1,00001.jpg,94,94
2,00002.jpg,166,166
3,00003.jpg,124,124
4,00004.jpg,189,189


In [16]:
from skimage import io
from skimage.transform import resize

In [17]:
from tqdm import tqdm_notebook

In [18]:
images = []
shapes = []
NEW_SHAPE = 100
for row in tqdm_notebook(size_data.iterrows()):
    im = io.imread("00_input/train/images/{}".format(row[1]['img_filename']))
    images.append(resize(im, (NEW_SHAPE, NEW_SHAPE, 3)))
    shapes.append((row[1]['n_rows'], row[1]['n_cols']))
images = np.array(images)
shapes = np.array(shapes)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

/home/ya_philya/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [19]:
points = []
for i in range(shapes.shape[0]):
    point = []
    for col in points_data.columns:
        if col.startswith('y'):
            point.append(float(points_data.iloc[i][col]) / float(shapes[i][0]) * float(NEW_SHAPE))
        if col.startswith('x'):
            point.append(float(points_data.iloc[i][col]) / float(shapes[i][1]) * float(NEW_SHAPE))
    points.append(point)
points = np.array(points, dtype=np.float64)
points.shape

(6000, 28)

In [20]:
points[:5]

array([[19.46564885, 19.46564885, 48.47328244, 22.90076336, 64.50381679,
        24.80916031, 77.48091603, 19.84732824, 25.95419847, 29.38931298,
        32.44274809, 28.6259542 , 41.98473282, 31.29770992, 61.06870229,
        33.20610687, 65.26717557, 30.15267176, 73.66412214, 32.0610687 ,
        59.54198473, 54.96183206, 35.11450382, 66.41221374, 53.4351145 ,
        68.70229008, 64.1221374 , 68.32061069],
       [22.34042553, 21.27659574, 38.29787234, 19.14893617, 60.63829787,
        20.21276596, 85.10638298, 26.59574468, 24.46808511, 28.72340426,
        36.17021277, 27.65957447, 42.55319149, 29.78723404, 62.76595745,
        31.91489362, 72.34042553, 31.91489362, 82.9787234 , 35.10638298,
        45.74468085, 48.93617021, 32.9787234 , 64.89361702, 47.87234043,
        70.21276596, 63.82978723, 69.14893617],
       [13.85542169, 69.87951807, 23.4939759 , 50.        , 28.31325301,
        34.3373494 , 33.73493976, 11.44578313, 21.68674699, 69.27710843,
        24.09638554, 57.8313

In [21]:
model = keras.models.Sequential()
model.add(L.Convolution2D(filters=16, kernel_size=3, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Convolution2D(filters=32, kernel_size=3, activation='relu'))
model.add(L.MaxPooling2D())
model.add(L.BatchNormalization())

model.add(L.Convolution2D(filters=64, kernel_size=3, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Convolution2D(filters=128, kernel_size=3, activation='relu'))
model.add(L.MaxPooling2D())
model.add(L.BatchNormalization())

model.add(L.Flatten())
model.add(L.Dense(units=1024, activation='relu'))
model.add(L.Dropout(rate=0.4))
model.add(L.Dense(units=28))
model.compile('adam', loss='mse')
model.build((None, NEW_SHAPE, NEW_SHAPE, 3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
batch_normalization_5 (Batch (None, 98, 98, 16)        64        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 96, 96, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 46, 46, 64)        18496     
_________________________________________________________________
batch_normalization_7 (Batch (None, 46, 46, 64)        256       
__________

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_test, y_train, y_test, shape_train, shape_test = train_test_split(images, points, shapes, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape, shape_train.shape, shape_test.shape

((4800, 64, 64, 3),
 (1200, 64, 64, 3),
 (4800, 28),
 (1200, 28),
 (4800, 2),
 (1200, 2))

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=20, verbose=2, validation_data=(X_test, y_test))

Train on 4800 samples, validate on 1200 samples
Epoch 1/20
 - 149s - loss: 81.7155 - val_loss: 27.3306
Epoch 2/20
 - 157s - loss: 36.6864 - val_loss: 48.3448
Epoch 3/20
 - 154s - loss: 26.6482 - val_loss: 17.9013
Epoch 4/20
 - 181s - loss: 22.0866 - val_loss: 9.9611
Epoch 5/20
 - 164s - loss: 18.4939 - val_loss: 9.3166
Epoch 6/20
 - 171s - loss: 16.5560 - val_loss: 8.5025
Epoch 7/20
 - 165s - loss: 14.6491 - val_loss: 14.6644
Epoch 8/20
 - 164s - loss: 13.9241 - val_loss: 9.5481
Epoch 9/20
 - 162s - loss: 13.5204 - val_loss: 13.4971
Epoch 10/20
 - 158s - loss: 13.4930 - val_loss: 8.5732
Epoch 11/20
 - 156s - loss: 12.5968 - val_loss: 8.2740
Epoch 12/20
 - 164s - loss: 12.0596 - val_loss: 7.1316
Epoch 13/20
 - 173s - loss: 11.4595 - val_loss: 6.7988
Epoch 14/20
 - 173s - loss: 11.0778 - val_loss: 6.9523
Epoch 15/20
 - 168s - loss: 11.2037 - val_loss: 5.7043
Epoch 16/20
 - 167s - loss: 11.6340 - val_loss: 6.4182
Epoch 17/20
 - 165s - loss: 10.8301 - val_loss: 5.7175
Epoch 18/20
 - 167s -

In [5]:
import os
os.listdir

<function posix.listdir>

In [23]:
model.predict(images[0][np.newaxis])

array([[ 0.00908441,  0.04967747,  0.0149589 , -0.02984657, -0.01538389,
         0.06967603,  0.03703532, -0.0288459 , -0.00062133, -0.01685303,
         0.01175428,  0.06271955, -0.05578396,  0.02015104,  0.02288132,
        -0.02077639, -0.00331962, -0.00873328,  0.0234977 , -0.03772496,
        -0.01603477, -0.01858955,  0.02337012, -0.01154331, -0.05542017,
         0.0226067 , -0.05349622,  0.04863688]], dtype=float32)

In [11]:
!python script.py .

/home/ya_philya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ya_philya/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
2018-10-27 16:30:41.627653: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-10-27 16:30:44.095009: W tensorflow/core/framework/allocator.cc:101] Allocation of 78675968 exceeds 10% of system memory.
2018-10-27 16:30:44.225756: W tensorflow/core/framework/allocator.cc:101] Allocation of 78675968 exceeds 10

In [5]:
list({'1':1, '2':2}.items())

[('1', 1), ('2', 2)]